# Fresh take

## 1. Initalize sites

In [ ]:
import os
import os
import shutil
import dbPipeline
import importlib
importlib.reload(dbPipeline)

projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass

db = dbPipeline.database(projectPath=projectPath)
# dbPipeline.projectInventory(projectPath=projectPath)

Sites = r'example_data\projectInventory_Template.yml'
db.projectInventory()
db.projectInventory(Sites=Sites)
db.projectInventory(Sites = {'FIL':{'siteID':'FIL'}})

c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\dbPipeline.py line 57:
{'SCL': {'siteID': 'SCL', 'description': 'Eddy covariance site in a permafrost wetland in the outer Mackenzie River Delta', 'Name': 'Swiss Cheese Lake', 'PI': 'June Skeeter, Peter Morse', 'startDate': datetime.date(2024, 7, 20), 'stopDate': None, 'landCoverType': 'Sedge Wetland', 'latitude': 69.2098, 'longitude': -135.2519067, 'Measurements': {'Flux': {'measurementID': 'Flux', 'fileType': 'TOBA', 'sampleFrequency': '50ms', 'description': 'High frequency (20 Hz) Eddy Covariance data', 'latitude': 69.2098, 'longitude': -135.2519067, 'startDate': datetime.date(2024, 7, 20), 'stopDate': None}, 'SSM_Profile': {'measurementID': 'SSM_Profile', 'fileType': 'HOBOcsv', 'sampleFrequency': '30m', 'description': 'HOBO logger in salix sedge meadow', 'latitude': 69.2253983, 'longitude': -135.2522633, 'startDate': datetime.date(2024, 7, 20), 'stopDate': None}, 'WSM_Profile': {'measurementID': 'WSM_Profile', 'fileType': 'HOBOcsv', 

In [752]:
importlib.reload(dbPipeline)

fpath = r'example_data\projectInventory_Template.yml'
sourcePath = r'example_data'
dbPipeline.projectInventory(
    projectPath=projectPath,Sites=fpath,
    fileSearch={'SCL':{'Flux':{'searchInventory':{'sourcePath':sourcePath,'wildcard':'*Flux*.dat'}},
                       'SSM_Profile':{'searchInventory':{'sourcePath':sourcePath,'wildcard':'*SSM*.csv'}},
                       'WSM_Profile':{'searchInventory':{'sourcePath':sourcePath,'wildcard':'*WSM*.csv'}},
                       }}
)

projectInventory(projectPath='c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\test', fillChar='_', sepChar='//', verbose=False, projectInfo={'dateCreated': '2025-03-20T15:38:53.712651', 'dateModified': '2025-03-20T15:38:53.759269', 'directoryStructure': {'database': {'.info': 'binary database files', 'defaultFormat': {'data': {'dtype': 'float32', 'info': 'data are stored as 32-bit floating point numbers'}, 'timestamp': {'dtype': 'float64', 'info': 'the POSIX_timestamp is stored as a 64-bit floating point number, representing the seconds elapsed since 1970-01-01 00:00 UTC time', 'timezone': 'UTC'}}}, 'metadata': {'.info': 'yaml files containing metadata'}}}, Sites={'SCL': {'siteID': 'SCL', 'description': 'Eddy covariance site in a permafrost wetland in the outer Mackenzie River Delta', 'Name': 'Swiss Cheese Lake', 'PI': 'June Skeeter, Peter Morse', 'startDate': datetime.date(2024, 7, 20), 'stopDate': None, 'landCoverType': 'Sedge Wetland', 'latitude': 69.2098, 'longitude': -135.25

In [763]:
importlib.reload(dbPipeline)
f = dbPipeline.database(projectPath=projectPath)
f.projectInventory(Sites = {'FIL':{'siteID':'FIL'}})


c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\dbPipeline.py line 64:
{'FIL': {'siteID': 'FIL', 'description': None, 'Name': None, 'PI': None, 'startDate': None, 'stopDate': None, 'landCoverType': None, 'latitude': None, 'longitude': None, 'Measurements': {'measurementID': {'measurementID': 'measurementID', 'fileType': 'fileType', 'sampleFrequency': 'sampleFrequency', 'description': 'description', 'latitude': 0.0, 'longitude': 0.0, 'startDate': 'startDate', 'stopDate': 'stopDate'}}}}



In [665]:
import os
import shutil
import cleanDev as cD
import importlib
importlib.reload(cD)

projectPath = os.path.join(os.getcwd(),'test')
# try:
#     shutil.rmtree(projectPath)
# except:
#     pass

fpath = r'example_data\projectInventory_Template.yml'
sourcePath = r'example_data'
cD.projectInventory(
    projectPath=projectPath,Sites=fpath,
    # fileSearch={'SCL':{'Flux':{'searchInventory':{'sourcePath':sourcePath,'wildcard':'*Flux*.dat'}}}}
        # 'SCL':{
        #     'SSM_Profile':cD.Source(sourcePath=sourcePath,fileExt='csv',matchPattern=['SSM','readout'],fileType='HOBOcsv').__dict__,
        #     'WSM_Profile':cD.Source(sourcePath=sourcePath,fileExt='csv',matchPattern=['WSM','readout'],fileType='HOBOcsv').__dict__,
        #     'Flux':cD.Source(sourcePath=sourcePath,fileExt='dat',matchPattern=['Flux'],fileType='TOBA').__dict__}}
)


projectInventory(projectPath='c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\test', fillChar='_', sepChar='//', verbose=False, projectInfo={'dateCreated': '2025-03-20T13:41:35.964147', 'dateModified': '2025-03-20T14:27:26.493351', 'directoryStructure': {'database': {'.info': 'binary database files', 'defaultFormat': {'data': {'dtype': 'float32', 'info': 'data are stored as 32-bit floating point numbers'}, 'timestamp': {'dtype': 'float64', 'info': 'the POSIX_timestamp is stored as a 64-bit floating point number, representing the seconds elapsed since 1970-01-01 00:00 UTC time', 'timezone': 'UTC'}}}, 'metadata': {'.info': 'yaml files containing metadata'}}}, Sites={'SCL': {'siteID': 'SCL', 'description': 'Eddy covariance site in a permafrost wetland in the outer Mackenzie River Delta', 'Name': 'Swiss Cheese Lake', 'PI': 'June Skeeter, Peter Morse', 'startDate': datetime.date(2024, 7, 20), 'stopDate': None, 'landCoverType': 'Sedge Wetland', 'latitude': 69.2098, 'longitude': -135.25

In [245]:
importlib.reload(cD)
# cD.searchInventory(
#     projectPath=projectPath,
#     siteID='SCL',
#     measurementID='Flux',
#     searchInventory={
#     'sourcePath':r'example_data',
#     'fileType':'HOBOcsv',
#     'fileExt':'csv',
#     'matchPattern':['20240724','SSM','readout']
#     })
cD.searchInventory(
    projectPath=projectPath,
    siteID='SCL',
    measurementID='Flux',
    searchInventory={
    'sourcePath':r'example_data',
    'wildcard':'*Flux*.dat',
    })

line 177: {'example_data\\*Flux*.dat': {'sourcePath': 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data', 'wildcard': '*Flux*.dat', 'nFiles': 5, 'nLoaded': 0, 'nPending': 5, 'fileList': ['20240912\\Flux_Data2628.dat', '20240912\\Flux_Data2629.dat', '20240912\\Flux_Data2630.dat', '20240912\\Flux_Data2631.dat', '20240912\\Flux_Data2632.dat'], 'loadList': [False, False, False, False, False]}}
line 179: example_data\*Flux*.dat


searchInventory(projectPath='c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\test', siteID='SCL', measurementID='Flux', searchInventory={'example_data\\*Flux*.dat': {'sourcePath': 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data', 'wildcard': '*Flux*.dat', 'nFiles': 5, 'nLoaded': 0, 'nPending': 5}})

In [227]:
import fnmatch
sourcePath=r'example_data'
sourcePath=os.path.abspath(sourcePath)
for dir,_,fn in os.walk(sourcePath):
   #  subDir = os.path.relpath(dir,sourcePath)
    print([os.path.join(dir,f) for f in fn 
           if fnmatch.fnmatch(os.path.join(sourcePath,f),os.path.join(sourcePath+'*'))
        #    and not fnmatch.fnmatch(os.path.join(subDir,f),'202409*SSM*readout*.csv')
           ])

['c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\projectInventory_Template.yml', 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\siteInventory.yml']
['c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240724\\20750527-SHSC.WSM.SGT.240720_240724readout.csv', 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240724\\20750528-SHSC.SSM.SGT.240720_240724readout.csv']
['c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240912\\Flux_Data2628.dat', 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240912\\Flux_Data2629.dat', 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240912\\Flux_Data2630.dat', 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240912\\Flux_Data2631.dat', 'c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240912\\Flux_Data2632.dat']
['c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\example_data\\20240914\\20750527-SHSC.WSM.SGT.240720_240913readout.csv', 'c:\\Users\\User\\GSC_Work\

In [320]:

cD.Search().uID

'uID 2025-03-19T16:22:45'

In [58]:
from dataclasses import dataclass, field
import helperFunctions as helper
@dataclass
class fileSearch:
    sourceID: str = 'sourceID'
    fileType: str = None
    fileExt: str = None
    sourcePath: str = None
    matchPattern: list = field(default_factory=lambda:[])
    excludePattern: list = field(default_factory=lambda:[])
    
    def __post_init__(self):
        self.sourceID = helper.safeFmt(self.sourceID)
        for f,v in self.__dataclass_fields__.items():
            if v.type is list:
                if type(self.__dict__[f]) is not list:
                    self.__dict__[f] = [self.__dict__[f]]
        if self.sourcePath:
            self.sourcePath = os.path.abspath(self.sourcePath)
        pass

@dataclass
class sourceFiles:
    siteID: str
    measurementID: str
    sourceFiles: fileSearch

    def __post_init__(self):
        if type(list(self.sourceFiles.values())[0]) is dict:
            self.sourceFiles = {fileSearch(**value).sourceID:fileSearch(**value).__dict__ for key,value in self.sourceFiles.items()}
        else:
            self.sourceFiles = {fileSearch(**self.sourceFiles).sourceID:fileSearch(**self.sourceFiles).__dict__}

data = {'siteID': 'SCL', 'measurementID': 'Flux', 'sourceFiles': {'sourceID':'X'}}

print(sourceFiles(**data))

sourceFiles(siteID='SCL', measurementID='Flux', sourceFiles={'X': {'sourceID': 'X', 'fileType': None, 'fileExt': None, 'sourcePath': None, 'matchPattern': [], 'excludePattern': []}})


In [92]:
import os
import shutil
import importlib
import NewDev as ND
import helperFunctions
importlib.reload(ND)  
importlib.reload(helperFunctions)   

# # fresh database
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass

sites = helperFunctions.loadDict('example_data/siteInventory.yml')
for site in sites.values():
    ND.siteInventory(projectPath=projectPath,**site)


# source = r'example_data'
# ND.measurementInventory(projectPath=projectPath,
#   siteID='SCL',
#   measurementID='Soil Profile',
#   pointID='SSM',
#   fileType='HOBOcsv',
#   baseFrequency='30min',
#   description = 'Soil temperature profiles in the salix sedge meadow',
#   sourceInventory={
#     '20240724':{
#         'sourcePath':source,
#         'sourceID':'20240724',
#         'matchPattern':['20240724','SSM','readout'],
#         'fileExt':'csv'},
#     '20240914':{
#         'sourcePath':source,
#         'sourceID':'20240914',
#         'matchPattern':['20240914','SSM','readout'],
#         'fileExt':'csv'},
#   })

SCL
['siteID', 'description', 'Name', 'PI', 'startDate', 'stopDate', 'landCoverType', 'latitude', 'longitude', 'samplePoints']
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\metadata\SCL\samplePoints.yml
Saving:  siteInventory.yml
ILL
['siteID', 'description', 'Name', 'PI', 'startDate', 'stopDate', 'landCoverType', 'latitude', 'longitude', 'samplePoints']
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\metadata\ILL\samplePoints.yml
Saving:  siteInventory.yml
FIL
['siteID', 'description', 'Name', 'PI', 'startDate', 'stopDate', 'landCoverType', 'latitude', 'longitude', 'samplePoints']
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\metadata\FIL\samplePoints.yml
Saving:  siteInventory.yml


## 2. Define measurments

In [46]:

ND.measurementInventory(projectPath=projectPath,
  siteID='SCL',
  measurementID='Flux',
  fileType='TOBA',
  baseFrequency='50ms',
  measurementDescription = 'High frequency eddy covariance data'
  )
ND.measurementInventory(projectPath=projectPath,
  siteID='SCL',
  measurementID='Meteorologic',
  fileType='TOBA',
  baseFrequency='30min',
  measurementDescription = 'Low frequency meteorological data'
  )
ND.measurementInventory(projectPath=projectPath,
  siteID='SCL',
  measurementID='Metadata',
  fileType='TOBA',
  baseFrequency='24h',
  measurementDescription = 'Daily metadata summaries'
  )
ND.measurementInventory(projectPath=projectPath,
  siteID='SCL',
  measurementID='Soil Profile',
  subsiteID='SSM',
  fileType='HOBOcsv',
  baseFrequency='30min',
  measurementDescription = 'Soil temperature profiles in the salix sedge meadow'
  )

a = ND.measurementInventory(projectPath=projectPath,
  siteID='SCL',
  measurementID='Soil Profile',
  subsiteID='WSM',
  fileType='HOBOcsv',
  baseFrequency='30min',
  measurementDescription = 'Soil temperature profiles in the wet sedge meadow'
  )
# a.__dict__

In [48]:
# importlib.reload(ND)
source = r'example_data\20240724'
# t = ND.searchInventory(projectPath=projectPath,
#     siteID='SCL',
#     measurementID='Soil_Profile',
#     subsiteID='SSM',
#     fileType='HOBOcsv',
#     sourcePath=source,
#     fileExt='csv',
#     matchPattern=['SSM','readout']
#     )

# t = ND.searchInventory(projectPath=projectPath,
#     siteID='SCL',
#     measurementID='Soil_Profile',
#     subsiteID='WSM',
#     fileType='HOBOcsv',
#     sourcePath=source,
#     fileExt='csv',
#     matchPattern=['WSM','readout']
#     )

# t.findNewSourceFiles()
print(os.path.abspath(source))

c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\example_data\20240724


In [39]:
importlib.reload(ND)
ND.database(verbose=True,projectPath=projectPath).validate()
# ND.siteInventory(projectPath=projectPath)
# any([j==1 for j in [1,2,3]])

Validating metadta in:  c:\Users\User\GSC_Work\EC_dataPipeline\test


In [ ]:
# importlib.reload(ND)
# source = r'example_data\20240724'
# t = ND.sourcefileInventory(projectPath=projectPath,
#     siteID='SCL',
#     index='Soil_Profile-WSM-HOBOcsv',
#     sourcePath=source,
#     fileExt='csv',
#     matchPattern='SSsM')
# t = ND.sourcefileInventory(projectPath=projectPath,
#     siteID='SCL',
#     index='Soil_Profile-WSM-HOBOcsv',
#     sourcePath=source,
#     fileExt='csv',
#     matchPattern='SSsM')

# source = r'example_data\20240914'
# t = ND.sourcefileInventory(projectPath=projectPath,
#     siteID='SCL',
#     index='Soil_Profile-WSM-HOBOcsv',
#     sourcePath=source,
#     fileExt='csv',
#     matchPattern='SSM')
# # t.subIndex

AttributeError: module 'NewDev' has no attribute 'sourcefileInventory'

In [454]:
# 
# tuple(['a','b'])
tuple(['ab'])

('ab',)

## Setup an inventory

In [27]:
import os
import yaml
import time
import shutil
import numpy as np
import pandas as pd
import importlib
import NewDev as ND
import helperFunctions as hf
importlib.reload(ND)     

# fresh database
projectPath = os.path.join(os.getcwd(),'test')
try:
    shutil.rmtree(projectPath)
except:
    pass
# ND.databaseProject(projectPath)

# load templates
template = hf.loadDict(r'example_data\siteInventory.yml')
for siteID,values in template.items():
    print(values)
    ND.siteInventory(projectPath=projectPath,**values)

template = hf.loadDict(r'example_data\SCL\measurementInventory.yml')
for measurementID,values in template.items():
    ND.measurementInventory(projectPath=projectPath,measurementID=measurementID,siteID='SCL',**values)

source = r'example_data'
ND.fileInventory(projectPath=projectPath,source=source,siteID='SCL',measurementID='Profile_SSM',ext='csv',matchPattern='SSM')#,lookup=True)
ND.fileInventory(projectPath=projectPath,source=source,siteID='SCL',measurementID='Profile_WSM',ext='csv',matchPattern='WSM')#,lookup=True)

{'siteID': 'SCL', 'siteDescription': 'Eddy covariance site in a permafrost wetland in the outer Mackenzie River Delta', 'Name': 'Swiss Cheese Lake', 'PI': 'June Skeeter, Peter Morse', 'startDate': '2024-07-20', 'stopDate': None, 'landCoverType': 'Sedge Wetland', 'latitude': 69.3722778, 'longitude': -134.8810889, 'subSites': {'SSM': {'subsiteID': 'SSM', 'description': 'Salix sedge meadow', 'latitude': 69.3722778, 'longitude': -134.8810889}, 'WSM': {'subsiteID': 'WSM', 'description': 'Wet sedge meadow', 'latitude': 69.3722778, 'longitude': -134.8810889}}}
Saving:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\metadata\SCL\subSites.yml
Saving:  siteInventory.yml
{'siteID': 'ILL', 'Name': 'Illisarvik', 'PI': 'Chris Burn', 'siteDescription': "Eddy Covariance site in an artificially drained thermokarst lake basin on Richard's Island", 'stopDate': '2016-08-07', 'landCoverType': 'Low Shrub/Meadow', 'latitude': 69.4798611, 'longitude': 'W 134 50.8385278', 'startDate': '2016-07-08'}
Savi

TypeError: measurementInventory.__init__() got an unexpected keyword argument 'description'

In [429]:
# ND.fileInventory(projectPath=projectPath,siteID='SCL',measurementID='Profile_WSM',lookup=True).sessionMetadata
# # a = None
# # if a:
# #     print('d')
d={['1','2']:'x'}

TypeError: unhashable type: 'list'

## File Management

In [ ]:
import time
import rawDataFile as rd
importlib.reload(rd)
source = r'example_data'
T1 = time.time()

t = rd.fileInventory(projectPath=projectPath,source=source,siteID='SCL',measurementID='Profile_SSM',ext='csv',matchPattern='SSM',lookup=True)

t.inventory

{'c:\\Users\\jskeeter\\gsc-permafrost\\EC_dataPipeline\\example_data': {'ext': 'csv',
  'matchPattern': ['SSM'],
  'excludePattern': [],
  'fileList': [['20240724\\20750528-SHSC.SSM.SGT.240720_240724readout.csv',
    False],
   ['20240914\\20750528-SHSC.SSM.SGT.240720_240913readout.csv', False]]}}

In [ ]:
import pandas as pd
pd.date_range('2025-03-01','2025-03-02',freq='50ms')

DatetimeIndex([       '2025-03-01 00:00:00', '2025-03-01 00:00:00.050000',
               '2025-03-01 00:00:00.100000', '2025-03-01 00:00:00.150000',
               '2025-03-01 00:00:00.200000', '2025-03-01 00:00:00.250000',
               '2025-03-01 00:00:00.300000', '2025-03-01 00:00:00.350000',
               '2025-03-01 00:00:00.400000', '2025-03-01 00:00:00.450000',
               ...
               '2025-03-01 23:59:59.550000', '2025-03-01 23:59:59.600000',
               '2025-03-01 23:59:59.650000', '2025-03-01 23:59:59.700000',
               '2025-03-01 23:59:59.750000', '2025-03-01 23:59:59.800000',
               '2025-03-01 23:59:59.850000', '2025-03-01 23:59:59.900000',
               '2025-03-01 23:59:59.950000',        '2025-03-02 00:00:00'],
              dtype='datetime64[ns]', length=1728001, freq='50ms')

In [ ]:
# fn
import os
import rawDataFile as rawDataFile
importlib.reload(rawDataFile)
# fn = fi['SCL']['TS_Profile-WSM']['c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024']['sourceFiles'][0]
# fn = os.path.join('c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024',fn)
fn = r'c:\\Users\\jskeeter\\gsc-permafrost\\SCL_2024\\20240724\\20750527-SHSC.WSM.SGT.240720_240724readout.csv'
rd = rawDataFile.hoboCSV(projectPath=projectPath,source=fn,siteID='SCL',measurementType='TS_Profile',loggerID='WSM',
                         Metadata='test.yml')

with open('test.yml','w+') as f:
    yaml.safe_dump(rd.Metadata,f,sort_keys=False)

{'TS_1_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 5)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 5 cm depth', 'vertical': 1, 'horizontal': 1, 'replicate': 1}, 'TS_2_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 25)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 25 cm depth', 'vertical': 2, 'horizontal': 1, 'replicate': 1}, 'TS_3_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 50)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temperature at 50 cm depth', 'vertical': 3, 'horizontal': 1, 'replicate': 1}, 'TS_4_1_1': {'variableName': 'TS', 'originalName': 'Temp, °C (LGR S/N: 20750527, SEN S/N: 20750527, LBL: 100)', 'ignore': False, 'unit': 'deg C', 'dtype': '<f8', 'variableDescription': 'Soil Temper

# Database

In [ ]:
importlib.reload(ND)

# projectPath = os.path.join(os.getcwd(),'test2')
# # if os.path.isdir(projectPath):
# #     shutil.rmtree(projectPath)

# db = pyDB.database(projectPath=projectPath)
# # db = pyDB.database(projectPath=projectPath,
# #     # siteID=['SCL'],
# #     # Years=[str(y) for y in range(2024,2025)],
# #     # verbose=False
# #     )
# # db.
n = ND.rawDatabaseImport(projectPath=projectPath,measurementID ='test',dataIn=rd.Data,metadataIn=rd.Metadata)

n

_ {'ignore': True, 'name_in': '#', 'unit_in': None, 'safe_name': '_', 'dtype': '<i8', 'variableDescription': None}
Date_Time_GMT_00_00 {'ignore': True, 'name_in': 'Date Time, GMT+00:00', 'unit_in': None, 'safe_name': 'Date_Time_GMT_00_00', 'dtype': '|O', 'variableDescription': None}
Temp_C_5cm {'ignore': False, 'name_in': 'Temp, °C (5cm', 'unit_in': None, 'safe_name': 'Temp_C_5cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_25cm {'ignore': False, 'name_in': 'Temp, °C (25cm', 'unit_in': None, 'safe_name': 'Temp_C_25cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_50cm {'ignore': False, 'name_in': 'Temp, °C (50cm', 'unit_in': None, 'safe_name': 'Temp_C_50cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_100cm {'ignore': False, 'name_in': 'Temp, °C (100cm', 'unit_in': None, 'safe_name': 'Temp_C_100cm', 'dtype': '<f8', 'variableDescription': None}


rawDatabaseImport(projectPath='c:\\Users\\User\\GSC_Work\\EC_dataPipeline\\test', overwrite=False, verbose=True, logFile='Creating Database: 2025-03-01 21:22:42\n', metadataFile={'defaultFormat': {'data': {'dtype': 'float32', 'info': 'data are stored as 32-bit floating point numbers'}, 'POSIX_timestamp': {'dtype': 'float64', 'timezone': 'UTC', 'info': 'the POSIX_timestamp is stored as a 64-bit floating point number, representing the seconds elapsed since 1970-01-01 00:00 UTC time'}}, 'Date_created': '2025-03-01 21:22:42', 'Last_modified': '2025-03-01 21:22:42'}, measurementID='test', dataIn=                        _ Date_Time_GMT_00_00  Temp_C_5cm  Temp_C_25cm  \
2024-07-20 21:00:00     1   24/07/20 21:00:00       7.217        2.423   
2024-07-20 21:30:00     2   24/07/20 21:30:00      11.759        6.712   
2024-07-20 22:00:00     3   24/07/20 22:00:00      11.467        5.257   
2024-07-20 22:30:00     4   24/07/20 22:30:00      11.029        4.428   
2024-07-20 23:00:00     5   24/0

In [ ]:

# siteFields = set(self.__dataclass_fields__) - {f for base in siteRecord.__bases__ if hasattr(base,'__dataclass_fields__') for f in base.__dataclass_fields__}
# siteFields = set(self.__dataclass_fields__) - {f for base in origin.__bases__ if hasattr(base,'__dataclass_fields__') for f in base.__dataclass_fields__}


def baseFields(self,repr=True,orderd=True):
    X = (set(self.__dataclass_fields__) - {f for base in type(self).__bases__ if hasattr(base,'__dataclass_fields__') for f,v in base.__dataclass_fields__.items() if v.repr==repr})
    print(X)
    print([k for k in self.__dataclass_fields__ if k in X])


from dataclasses import dataclass, field
@dataclass
class A:
    b: int = field(default=2,repr=False)
    j: int = 1

@dataclass
class B(A):
    x: str = 'a'
    b: int = field(default=3,repr=False)
    y: str = 'a'

baseFields(B())

{'b', 'y', 'x'}
['b', 'x', 'y']
